In [1]:
#from utils.split import RandomSplitPt
#from utils.dataset import HemorrhageDataset
from data_aug.dataset_wrapper import BloodDataset_Test, DataLoader
from models.resnet import HemoResNet18
from utils.metrics import hemorrhage_metrics
#from utils.util import get_test_pt_id_list, prediction_to_csv

import torch.nn.functional as F
import torch
import numpy as np
import pandas as pd


In [2]:
import csv
import os
import numpy as np
import pandas as pd

def get_test_pt_id_list(test_pt_folder_path):
    return os.listdir(test_pt_folder_path)

def prediction_to_csv(pt_name_list, image_name_list, prediction, output_csv_name=None,threshold=0.5, to_kaggle=False, remove_defunct=True):
    assert output_csv_name
    prediction = np.array(prediction > threshold, dtype=int)
    prediction_df = pd.DataFrame({"dirname":pt_name_list,"ID":image_name_list,
                                "ich":prediction[:,0],"ivh":prediction[:,1],"sah":prediction[:,2],
                                "sdh":prediction[:,3],"edh":prediction[:,4]})
    prediction_df.to_csv(output_csv_name, index=False)
    if to_kaggle:
        out_kaggle_name = output_csv_name.replace(".csv", "_KAGGLE.csv")
        ta_convert(output_csv_name, out_kaggle_name)
        print(f"Save as {out_kaggle_name}")
        if remove_defunct:
            os.remove(output_csv_name)
    else:
        print(f"Save as {output_csv_name}")

    
def csv_to_kaggle_ver(csv_name, ignore_ta=True, remove_defunct=True):
    kaggle_csv_name = csv_name.replace(".csv", "_KAGGLE.csv")
    if ignore_ta:
        ta_convert(csv_name, kaggle_csv_name)
        if remove_defunct:
            os.remove(csv_name)

    else:
        os.system(f"python3 to_kaggle.py --pred_csv_path {csv_name} --out_csv_path {kaggle_csv_name}")

def ta_convert(in_csv, out_csv):
    kaggle_eval_classes = ['ich', 'ivh', 'sah', 'sdh', 'edh']
    with open(in_csv) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        output_rows = []
        for row_idx, row in enumerate(csv_reader):
            if row_idx == 0:
                continue
            for cls_idx, cls in enumerate(kaggle_eval_classes):
                ID_single = row[1].split('.')[0] + '_' + cls
                output_row = [ID_single, row[cls_idx + 2]]
                output_rows.append(output_row)

                
    with open(out_csv, mode='w') as csv_file:
        fieldnames = ['ID', 'prediction']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        writer.writeheader()
        for row in output_rows:
            writer.writerow({'ID': row[0], 'prediction': row[1]})


In [3]:
ch = 3

test_dataset = BloodDataset_Test(path="/media/disk1/aa/Blood_data/test/", ch=ch)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers = 8)


In [4]:
#device = "cuda:0"
device = "cuda:1"
checkpoint = "./checkpoints/resnet18/best.pth"
num_class = 5

model = HemoResNet18(in_channels=ch, n_classes=num_class)
model.load_state_dict(torch.load(checkpoint, map_location="cpu"))
model.to(device);

In [5]:
model.eval()
pt_name_list = []
image_name_list = []
prediction = []
with torch.no_grad():
    for i, (img, _dir, _fname) in enumerate(test_loader):
        print(f"Process {i+1} / {len(test_loader)}    ", end="\r")
        img = img.to(device)

        logits = model(img)
        logits = torch.sigmoid(logits)
        logits = logits.cpu().numpy()
        prediction.append(logits)
        pt_name_list += list(_dir)
        image_name_list += list(_fname)
prediction = np.concatenate((prediction))

In [6]:
prediction_to_csv(pt_name_list, image_name_list, prediction, 
                  output_csv_name="./submit.csv", 
                  threshold=0.5, to_kaggle=True, remove_defunct=True)

Save as ./submit_KAGGLE.csv


In [7]:
import pandas as pd


In [8]:
df = pd.read_csv('/media/disk1/aa/Blood_data/train.csv')
df

,dirname,ID,ich,ivh,sah,sdh,edh
0,ID_3bfb897858,3bfb897858_0.jpg,0,0,0,0,0
1,ID_3bfb897858,3bfb897858_1.jpg,0,0,0,0,0
2,ID_3bfb897858,3bfb897858_2.jpg,0,0,0,0,0
3,ID_3bfb897858,3bfb897858_3.jpg,0,0,0,0,0
4,ID_3bfb897858,3bfb897858_4.jpg,0,0,0,0,0
...,...,...,...,...,...,...,...
50487,ID_220713d6c7,220713d6c7_23.jpg,0,0,0,0,0
50488,ID_220713d6c7,220713d6c7_24.jpg,0,0,0,0,0
50489,ID_220713d6c7,220713d6c7_25.jpg,0,0,0,0,0
50490,ID_220713d6c7,220713d6c7_26.jpg,0,0,0,0,0
